<a href="https://www.kaggle.com/code/esatakkasoglu/astronomical-calculations?scriptVersionId=161060101" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Solar System Orbital Dynamics Notebook

## Introduction
This notebook is dedicated to exploring and calculating various aspects of orbital dynamics within our Solar System. It includes detailed calculations and models that help in understanding the movement and positions of celestial bodies in relation to the Sun. The notebook uses a combination of physical constants, celestial mechanics principles, and Kepler's laws to perform accurate calculations. It's designed to assist astronomers, students, and space enthusiasts in visualizing and comprehending the intricate dance of planets and other celestial bodies around the Sun.

## Example Calculation
Below is an example calculation demonstrating how this notebook can be used to determine the polar coordinates of a celestial body in the Solar System at a specific time relative to its orbit around the Sun. This example employs Kepler's laws and relevant astronomical data to find the position of a body in orbit.

### Question
Calculate the polar coordinates of a celestial body in the Solar System 150 days after the perihelion, given the following data:

- Perihelion distance (q): 2.5467 Astronomical Units (AU)
- Aphelion distance (Q): 2.9847 Astronomical Units (AU)
- Mass of the Sun (M_Sun): 2 x 10^33 grams
- Gravitational constant (G): 6.67 x 10^-8 dyn cm²/g²
- Required precision: After the comma, more then 8 digits in the solution of Kepler's equation

**Solution:**
- Semi-major axis (a): 2.7657 AU
- Eccentricity (e): 0.0792
- Orbital period (P): 1682.4133 days
- Mean anomaly (M): 32.0968 degrees
- Distance from Sun (r): 2.5856 AU
- True anomaly (f): 37.3514 degrees

This example showcases the method of calculating the position of a celestial body in its orbit at a given time using established astronomical and physical constants.


# Orbital Mechanics Calculation

This section outlines the step-by-step process to calculate the orbital parameters and position of a celestial body in its orbit around the Sun.

## Given Data

- Perihelion distance (q): 2.5467 AU
- Aphelion distance (Q): 2.9847 AU
- Mass of the Sun (M_Sun): 2 x 10^33 g
- Gravitational constant (G): 6.67 x 10^-8 dyn·cm²/g²

## Calculations

Given the perihelion distance $q = 2.5467$ AU, and the aphelion distance $Q = 2.9847$ AU, the semi-major axis $a$ is computed as:
$$ a = \frac{q + Q}{2} = \frac{2.5467 + 2.9847}{2} = 2.7657 \text{ AU} $$

The eccentricity $e$ is given by:
$$ e = \frac{a - q}{a} = \frac{2.7657 - 2.5467}{2.7657} = 0.0792 $$

The orbital period $P$ can be found using Kepler's third law:
$$ P = 2\pi \sqrt{\frac{a^3}{G M_{\odot}}} $$

Where $G$ is the gravitational constant and $M_{\odot}$ is the mass of the Sun.

Once $P$ is known, the mean motion $n$ can be calculated:
$$ n = \frac{360}{P} $$

Using the mean motion, the mean anomaly $M$ at a given time $t$ after perihelion can be found:
$$ M = n (t - t_0) $$

Given $M$, the eccentric anomaly $E$ can be calculated iteratively using Kepler's equation:
$$ M = E - e \sin E $$

Finally, the true anomaly $f$ and the distance $r$ from the Sun can be found:
$$ r = a(1 - e\cos E) $$

$$ f = 2 \arctan \left(\sqrt{\frac{1+e}{1-e}} \tan \frac{E}{2}\right) $$

This process will yield the polar coordinates of the celestial body in its orbit at a given time.


In [1]:
import numpy as np

# Constants
AU_TO_CM = 1.5e+13  # Astronomical unit to centimeters
G = 6.67e-8  # Gravitational constant in dyn*cm^2/g^2
M_SUN = 2e33  # Mass of the Sun in grams
DEG_TO_RAD = np.pi / 180  # Degrees to radians conversion
RAD_TO_DEG = 180 / np.pi  # Radians to degrees conversion

def calculate_orbital_elements(q, Q):
    """
    Calculates various orbital elements based on perihelion and aphelion distances.
    
    Parameters:
    q (float): Perihelion distance in astronomical units (AU).
    Q (float): Aphelion distance in astronomical units (AU).
    
    Returns:
    dict: Dictionary containing semi-major axis, eccentricity, orbital period, mean motion.
    """
    # Calculate semi-major axis in AU
    a = (q + Q) / 2
    
    # Calculate eccentricity
    e = (a - q) / a
    
    # Convert semi-major axis to cm for period calculation
    a_cm = a * AU_TO_CM
    
    # Calculate orbital period in seconds
    P_seconds = 2 * np.pi * np.sqrt(a_cm**3 / (G * M_SUN))
    
    # Convert orbital period to days
    P_days = P_seconds / (60 * 60 * 24)
    
    # Calculate mean motion in degrees per day
    n = 360 / P_days
    
    return {
        'semi_major_axis': a,
        'eccentricity': e,
        'orbital_period': P_days,
        'mean_motion': n
    }

# The provided function already calculates the orbital elements. 
# To see the results, we can simply call the function with example values and print the results.

# Example values for perihelion and aphelion distances of a hypothetical celestial body
q_example = 2.5467  # Perihelion distance in AU
Q_example = 2.9847  # Aphelion distance in AU

# Call the function with the example values
orbital_elements = calculate_orbital_elements(q_example, Q_example)

# Print the results
print(f"Semi-major axis (a): {orbital_elements['semi_major_axis']} AU")
print(f"Eccentricity (e): {orbital_elements['eccentricity']}")
print(f"Orbital period (P): {orbital_elements['orbital_period']} days")
print(f"Mean motion (n): {orbital_elements['mean_motion']} degrees/day")



Semi-major axis (a): 2.7657 AU
Eccentricity (e): 0.07918429330730009
Orbital period (P): 1682.4133437415503 days
Mean motion (n): 0.21397833138875927 degrees/day


In [2]:
def keplers_equation(M, e, tolerance=1e-4):
    """
    Solves Kepler's equation M = E - e*sin(E) for the eccentric anomaly E
    using the Newton-Raphson method.
    
    Parameters:
    M (float): Mean anomaly in degrees.
    e (float): Eccentricity of the orbit (dimensionless).
    tolerance (float): Tolerance for the convergence of the solution.
    
    Returns:
    float: Eccentric anomaly E in degrees.
    """
    # Convert M from degrees to radians
    M_rad = np.radians(M)
    
    # Initial guess for E is M when e is small
    E = M_rad if e < 0.8 else np.pi
    
    # Initialize difference for iteration
    delta = tolerance + 1
    while delta > tolerance:
        E_new = E - (E - e * np.sin(E) - M_rad) / (1 - e * np.cos(E))
        delta = abs(E_new - E)
        E = E_new
    
    # Convert E back to degrees
    return np.degrees(E)


def true_anomaly(E, e):
    """
    Calculates the true anomaly f from the eccentric anomaly E.
    
    Parameters:
    E (float): Eccentric anomaly in degrees.
    e (float): Eccentricity of the orbit (dimensionless).
    
    Returns:
    float: True anomaly f in degrees.
    """
    # Convert E from degrees to radians
    E_rad = np.radians(E)
    print(E)
    # Calculate true anomaly f in radians
    f_rad = 2 * np.arctan2(np.sqrt(1 + e) * np.sin(E_rad / 2),
                           np.sqrt(1 - e) * np.cos(E_rad / 2))
    
    # Convert f back to degrees
    return np.degrees(f_rad)


def distance_from_sun(a, E, e):
    """
    Calculates the distance r from the sun using the semi-major axis and the eccentric anomaly.
    
    Parameters:
    a (float): Semi-major axis in astronomical units (AU).
    E (float): Eccentric anomaly in degrees.
    e (float): Eccentricity of the orbit (dimensionless).
    
    Returns:
    float: Distance r in astronomical units (AU).
    """
    # Convert E from degrees to radians
    E_rad = np.radians(E)
    
    # Calculate distance r in AU
    r = a * (1 - e * np.cos(E_rad))
    
    return r


# Test function using given values
def test_orbital_calculations():
    # Given values from the user's problem
    n = 0.2139753334  # Mean motion in degrees per day
    t = 150  # Time after perihelion in days
    t0 = 0  # Time at perihelion (assuming t0 = 0 for simplicity)
    e = 0.0792  # Eccentricity (dimensionless)
    a = 2.7657  # Semi-major axis in AU

    # Mean anomaly M
    M = n * (t - t0)
    print(M)
    # Eccentric anomaly E
    E = keplers_equation(M, e)
    print(E)
    # True anomaly f
    f = true_anomaly(E, e)
    
    # Distance r
    r = distance_from_sun(a, E, e)
    
    print(f"Eccentric anomaly E: {E} degrees")
    print(f"True anomaly f: {f} degrees")
    print(f"Distance r: {r} AU")

# Run the test function
test_orbital_calculations()


32.09630001
34.67816964076346
34.67816964076346
Eccentric anomaly E: 34.67816964076346 degrees
True anomaly f: 37.35080447729489 degrees
Distance r: 2.5855672431614893 AU
